In [3]:
!pip install bs4==4.10.0 -y
!pip install nbformat==4.2.0
!mamba install html5lib==1.1 -y


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -y


'mamba' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)
url = "https://www.cbr.ru/eng/hd_base/mrrf/mrrf_7d/?UniDbQuery.Posted=True&UniDbQuery.From=01.2021&UniDbQuery.To=11.2024.html"
data  = requests.get(url).text
soup = BeautifulSoup(data, 'html.parser')

# Instead of directly accessing tbody, find the table first and then iterate through its rows
table = soup.find('table', {'class': 'data'})  # Assuming the table has a class 'data'
# Check if the table is found
if table:
    reserve_data = pd.DataFrame(columns=["Date", "Volume"])
    for row in table.find_all('tr')[1:]:  # Skip the header row
        col = row.find_all("td")
        # Check if the expected number of columns is present
        if len(col) >= 2:
            Date = col[0].text
            Volume = col[1].text
            reserve_data = pd.concat([reserve_data, pd.DataFrame({"Date":[Date], "Volume":[Volume]})], ignore_index=True)
        else:
            print("Skipping row with insufficient columns:", row) # Print debug information
else:
    print("Table not found on the page.")

C:\Users\user\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
